(Alexander Huber, Seminar Computer assisted mathematics, 31.05.2023, Prof. Florent Schaffhauser)

# Lanczos Algorithm

In [2]:
import random
import warnings
warnings.filterwarnings('ignore')
R = RealField(150)

def findOrthogonalVector(vv):
    dim = len(vv[0])
    
    for i in range(dim):
        ei = vector(CDF, [0]*dim); ei[i] = 1.0
        ei -= sum((v*ei)*v for v in vv)
        if ei != vector(CDF, [0]*dim):
            return ei/ei.norm()
    return vector(CDF, [0]*dim)

def tridiagonalMatrix(alpha, beta):
    n = len(alpha)
    TD = diagonal_matrix(R, alpha)
    for i in range(0,n-2):
        TD[i+1,i] = beta[i+1]
        TD[i,i+1] = beta[i+1]
    return TD

## Prerequisites

#### Lemma 1: Rayleigh-Quotient

Let $A \in \mathbb{R}^{n \times n}$ be a symmetrical matrix, $\lambda _{\min}, \lambda _{\max} \in \mathbb{R}$ the smallest and largest eigenvalues of $A$. Then:

$$\lambda _{\min} = \min_{x \neq 0}\mu(x)$$

$$\lambda _{\max} = \max_{x \neq 0}\mu(x)$$

where $\mu(x) \coloneqq\frac{\langle x,Ax \rangle}{\langle x,x \rangle}$ denotes the Rayleigh quotient of $A$. 

,
 

#### Corollary 2: Approximation by range of Eigenvectors

Let $\lambda_{1}\leq...\leq\lambda_{n}$ be the Eigenvalues of $A\in\mathbb{R}^{n\times n}, \eta_{1},...,\eta_{n}$ the respective Eigenvectors.
Then:

$$\lambda_{i} = \min_{x \in span(\eta_{i},...,\eta_{n})}\mu(x)\,=\,\max_{x \in span(\eta_{1},...,\eta_{i})}\mu(x). $$



In [3]:
# Function returning Rayleigh-Quotient
def rayleigh(A,x):
    return x.dot_product(A*x)//x.dot_product(x)

# Define symmetric Matrix with preknown Eigenvalues 0, 1, 3
A = Matrix(QQ, [[1,0,1],[0,1,1],[1,1,2]])
show(A)

[1 0 1]
[0 1 1]
[1 1 2]

In [4]:
# Create random vectors in Field Q^3 and compute their Rayleigh-Coefficients
space = VectorSpace(QQ, 3)
rq = []
vec = []
for i in range(1,1000):
    x = space.random_element()
    if x.dot_product(x) != 0.:
        rq.append(rayleigh(A,x))
        vec.append(x)

In [5]:
# Find Vectors corresponding to the minimum, middle and maximum value
minvec = vec[rq.index(min(rq))]
midvec = vec[rq.index(1)]
maxvec = vec[rq.index(max(rq))]

print("Minimum Rayleigh Coefficient:  " + str(min(rq)))
print("Maximum Rayleigh Coefficient:  " + str(max(rq)))

Minimum Rayleigh Coefficient:  0
Maximum Rayleigh Coefficient:  3


In [6]:
print("Vector corresponding to maximum value:  "+str(maxvec))
print("maxvec*A:    "+str((maxvec*A)))
print("\nVector corresponding to minimum value:  "+str(minvec))
print("minvec*A:    "+str((minvec*A)))



Vector corresponding to maximum value:  (-1/2, -1/2, -1)
maxvec*A:    (-3/2, -3/2, -3)

Vector corresponding to minimum value:  (1, 1, -1)
minvec*A:    (0, 0, 0)


#### Basic idea

##### Approximate Eigenvalues by solving the extreme value problem for the Rayleigh-Quotient on a sequence of Subspaces $V_{1}\subset V_{2}\subset ... \subset \mathbb{R}^{n}$



#### Lemma 3: Approximation with Krylov Spaces

Let $x\neq 0, k=1,2,....$ , let $V_{k}(x)\coloneqq span\{x,Ax,...,A^{k-1}x\}$ be a Krylov Subspace. 
Then:

$$\lambda_{min}^{(k)}\coloneqq \min_{y \in V_{k}(x)}\mu(y)\geq\lambda_{min}^{(k+1)}\geq  \lambda_{min}$$

$$\lambda_{max}^{(k)}\coloneqq \max_{y \in V_{k}(x)}\mu(y)\leq\lambda_{max}^{(k+1)}\leq \lambda_{max}$$.

## Concept of Lanczos' Method

Compute Orthonormal Basis $v_{1}, ... , v_{k}$ of $V_{k}(x)$ iteratively:

$$v_{0}\coloneqq 0, \,\, v_{1}\coloneqq \frac{x}{||x||_{2}},$$
$$\alpha_{k}\coloneqq\langle v_{k},Av_{k}\rangle,$$
$$w_{k+1}\coloneqq Av_{k}-\alpha_{k}v_{k}-\beta_{k}v_{k-1},$$
$$\beta_{k+1}\coloneqq ||w_{k+1}||_{2},$$
$$v_{k+1}\coloneqq \frac{w_{k+1}}{\beta_{k+1}}\,\,\,\,\,\, if\,\,\, \beta_{k+1}\neq0$$


In [7]:
def lanczos(iters, A):
    # Initiation
    n = A.nrows()
    alpha = vector(RDF, [0]*iters)
    beta = vector(RDF, [0]*(iters))
    Q = [vector(RDF, [0]*n) for _ in range(iters)]
    
    Q[0][0] = 1.0
    alpha[0] = Q[0]*A*Q[0]
    w = A*Q[0] - alpha[0]*Q[0]
    beta[0] = w.norm()
    if beta == 0:
        Q[1] = findOrthogonalVector((Q[0],))
    else:
        Q[1] = w/beta[0]
    alpha[1] = Q[1]*A*Q[1]    
    
    #Iteration
    for i in range(2,iters):
        w = A*Q[i-1]-alpha[i-1]*Q[i-1]-beta[i-2]*Q[i-2]
        beta[i-1] = w.norm()
        if beta[i-1] != 0:
            Q[i] = w/beta[i-1]
        else:
            Q[i] = findOrthogonalVector(Q[range(0,i)])
    alpha[iters-1] = (Q[iters-1]*A)*Q[iters-1]
    
    return({'alpha': alpha, 'beta': beta, 'basis': matrix(RDF,Q)})

###### Test method on matrix A, computing $Q_{k}\coloneqq [v_{1},...,v_{k}]$ and $T_{k} = \begin{pmatrix}\alpha_{1} & \beta_{2} & 0\\\beta_{2} & \alpha_{2} & \beta_{3}\\ 0 & \beta_{3} & \alpha_{3}\end{pmatrix}$

In [8]:
# Lanczos with 3 iterations
res_A = lanczos(3, A)

A_tri = tridiagonalMatrix(res_A['alpha'],res_A['beta'])
show(A_tri.n(digits=2))

[ 1.0  1.0 0.00]
[ 1.0  2.0 0.00]
[0.00 0.00  1.0]

###### With exact arithmetic, $Q_{n}AQ_{n}^{T} = T_{n}$ holds, where Q is an orthonormal matrix, thus $QQ^{T}=Id$

In [9]:
Q = res_A['basis']

show((Q*A*Q.transpose()).n(digits=2))
print("\n \n")
show((Q*Q.transpose()).n(digits=2))


[ 1.0  1.0 0.00]
[ 1.0  2.0  1.0]
[0.00  1.0  1.0]

[ 1.0 0.00 0.00]
[0.00  1.0 0.00]
[0.00 0.00  1.0]

###### Comparison between the exact Eigenvalues of A and T:

In [10]:
show(A_tri.eigenvalues())
show(A.eigenvalues())

[2.6180339887498948482045868343656381177203092, 1.0000000000000000000000000000000000000000000, 0.38196601125010515179541316563436188227969082]

[3, 1, 0]

#### Application on large sparse symmetric matrices

In [11]:
ms = MatrixSpace(R, 70)
random.seed(int(46))
M = ms.random_element(density=0.1)
M = M + M.transpose()

result = {}
for k in range(10,80,10):
    temp = lanczos(k,M)
    T_temp = tridiagonalMatrix(temp['alpha'],temp['beta'])
    result['k='+str(k)] = T_temp.eigenvalues()
    print("k = "+str(k)+":     "  +str(round(max(result['k='+str(k)]),6)))

print("\nMaximum Eigenvalue approximated by built-in function:    " + str(round(max(M.eigenvalues()),6)))

k = 10:     4.154545
k = 20:     4.403183
k = 30:     4.403388
k = 40:     4.403391
k = 50:     4.404257
k = 60:     4.40426
k = 70:     4.404346

Maximum Eigenvalue approximated by built-in function:    4.404059
